In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [3]:
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [4]:
data = pd.read_csv('Review.csv')
data.head()


,Review,Sentiment
0,Fast shipping but this product is very cheaply...,1
1,This case takes so long to ship and it's not e...,1
2,Good for not droids. Not good for iPhones. You...,1
3,The cable was not compatible between my macboo...,1
4,The case is nice but did not have a glow light...,1


In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     24999 non-null  object
 1   Sentiment  25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [6]:
data.dropna(inplace=True)


In [7]:
data.shape

(24999, 2)

In [8]:
data.loc[data['Sentiment']<=3,'Sentiment'] = 0

data.loc[data['Sentiment']>3,'Sentiment'] = 1


In [9]:
data.head() 

,Review,Sentiment
0,Fast shipping but this product is very cheaply...,0
1,This case takes so long to ship and it's not e...,0
2,Good for not droids. Not good for iPhones. You...,0
3,The cable was not compatible between my macboo...,0
4,The case is nice but did not have a glow light...,0


In [10]:
data.tail() 

,Review,Sentiment
24995,These cables (lightning) are far superior to m...,1
24996,This unit performs exactly as advertised. I u...,1
24997,I had the key cut at a local hardware store cu...,1
24998,I love my case I ordered from amazon. It wasn'...,1
24999,Awsome!!! I loved this little speaker. It has ...,1


In [17]:
data['Review'] = data['Review'].str.lower()
data.head()

,Review,Sentiment
0,fast shipping product cheaply made brought gra...,0
1,case take long ship even worth dont buy,0
2,good droids good iphones use feature watch iph...,0
3,cable wa compatible macbook iphone also connec...,0
4,case nice glow light im disappointed product d...,0


In [18]:
import re
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

# Apply the 'remove_urls' function to 'review' column
data['Review'] = data['Review'].apply(remove_urls)

data.head()

,Review,Sentiment
0,fast shipping product cheaply made brought gra...,0
1,case take long ship even worth dont buy,0
2,good droids good iphones use feature watch iph...,0
3,cable wa compatible macbook iphone also connec...,0
4,case nice glow light im disappointed product d...,0


In [19]:
import string
def remove_punctuation(text):
    # Using the string.punctuation constant to get all punctuations
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Apply the 'remove_punctuation' function to 'review' column
data['Review'] = data['Review'].apply(remove_punctuation)

# Print the updated DataFrame
data.head()

,Review,Sentiment
0,fast shipping product cheaply made brought gra...,0
1,case take long ship even worth dont buy,0
2,good droids good iphones use feature watch iph...,0
3,cable wa compatible macbook iphone also connec...,0
4,case nice glow light im disappointed product d...,0


In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to apply lemmatization to a string
def apply_lemmatization(text):
    tokens = nltk.word_tokenize(text)  # Tokenize the text into words
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Apply lemmatization to each word
    return ' '.join(lemmatized_tokens)  # Join the lemmatized tokens back into a single string

# Apply the 'apply_lemmatization' function to 'review' column
data['Review'] = data['Review'].apply(apply_lemmatization)

# Print the updated DataFrame
data.head()

,Review,Sentiment
0,fast shipping product cheaply made brought gra...,0
1,case take long ship even worth dont buy,0
2,good droids good iphones use feature watch iph...,0
3,cable wa compatible macbook iphone also connec...,0
4,case nice glow light im disappointed product d...,0


In [22]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)  # Tokenize the text into words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]  # Remove stop words
    return ' '.join(filtered_tokens)  # Join the filtered tokens back into a single string

# Apply the 'remove_stopwords' function to 'review' column
data['Review'] = data['Review'].apply(remove_stopwords)

data.head()

,Review,Sentiment
0,fast shipping product cheaply made brought gra...,0
1,case take long ship even worth dont buy,0
2,good droids good iphones use feature watch iph...,0
3,cable wa compatible macbook iphone also connec...,0
4,case nice glow light im disappointed product d...,0


In [23]:
data['Sentiment'].value_counts() 

0    15000
1     9999
Name: Sentiment, dtype: int64

In [24]:
data['Review'][0]

'fast shipping product cheaply made brought grandchild ipod would protected fell skin suitable protection good decoration therefore rated it1'

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=2500)
X = cv.fit_transform(data['Review'] ).toarray()


In [26]:
X[0].shape

(2500,)

In [27]:
from sklearn.model_selection import train_test_split
x_train ,x_test,y_train,y_test=train_test_split(X,data['Sentiment'],test_size=0.20,random_state=42)


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming you have already split your data into x_train, x_test, y_train, y_test

# Create an instance of Logistic Regression classifier
lg = LogisticRegression()

# Model fitting
lg.fit(x_train, y_train)

# Testing the model
pred = lg.predict(x_test)

# Model accuracy
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)


Accuracy: 0.817


In [29]:
import joblib


# Save the model
joblib.dump(lg, "lg.joblib")

# Load the saved model
loaded_model = joblib.load("lg.joblib")

In [30]:
x_test[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [31]:
def decode_sentiment(score):
    return "Positive" if score > 0.5 else "Negative"

# Assuming you have trained and loaded your Logistic Regression model as 'loaded_model'
# Assuming you have prepared your test data and stored it in 'x_test'

# Generate scores
scores = loaded_model.predict_proba(x_test)[:, 1]  # Get the probabilities of positive class (index 1)

# Decode sentiment labels
y_pred_1d = [decode_sentiment(score) for score in scores]


In [32]:
len(scores)
len(y_pred_1d)

5000

In [47]:
n="Works well but consumes more power"
n=remove_urls(n)
n=remove_punctuation(n)
n=apply_lemmatization(n)
n=remove_stopwords(n)
print(n)
x=cv.transform(n.split()).toarray()

Works well consumes power


In [48]:
x=x.reshape(1,-1)

In [49]:
x

array([[0., 0., 0., ..., 0., 0., 0.]])

In [50]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have the sequences in 'x' as a list of lists

# Set the maximum feature length
max_features = 2500

# Perform zero-padding on the sequences
x = pad_sequences(x, maxlen=max_features, padding='post', truncating='post')

In [51]:
lg.predict(x)

array([0], dtype=int64)

In [52]:
def decode_sentiment(score):
    return "Positive" if score > 0.5 else "Negative"

# Assuming you have trained and loaded your Logistic Regression model as 'loaded_model'
# Assuming you have prepared your test data and stored it in 'x_test'

# Generate scores
scores = loaded_model.predict_proba(x)[:, 1]  # Get the probabilities of positive class (index 1)

# Decode sentiment labels
y_pred_1d = [decode_sentiment(score) for score in scores]


In [53]:
y_pred_1d

['Negative']